1. Activate the virtual environment: 
source /Users/sofiamorenolasa/Desktop/TFG/.venv/bin/activate
2. Switch to the virtual environment

In [17]:
import pandas as pd

file_path = "../data/PARAMETROS_TJ2.csv"

# Use latin1 encoding
data = pd.read_csv(file_path, delimiter=";", encoding="latin1")

# Display the first few rows
print(data.head())

   N_DESCARGA       fecha   hora comentarioDesc comentarioExp configuracion  \
0         112  19971217.0  19:05            NaN           NaN           NaN   
1         113  19971217.0  19:09            NaN           NaN           NaN   
2         114  19971217.0  19:55            NaN           NaN           NaN   
3         115  19971218.0  11:08            NaN           NaN           NaN   
4         116  19971218.0  11:28            NaN           NaN           NaN   

   potencia_radiada  energia_diamagnetica  retraso_densidad_girotron  zeff  \
0               NaN                   NaN                        NaN   NaN   
1               NaN                   NaN                        NaN   NaN   
2               NaN                   NaN                        NaN   NaN   
3               NaN                   NaN                        NaN   NaN   
4               NaN                   NaN                        NaN   NaN   

   ...  IAccel_nominal_NBI2  tini_NBI2  longitud_pulso_n

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/2676448733.py:6: DtypeWarning: Columns (52,55,96,132,133,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter=";", encoding="latin1")


Next, we sort the PARAMETROS_TJ2.csv file by the N_DESCARGA field and save the sorted data to a new file named PARAMETROS_TJ2_ORDENADOS.csv. This ensures the rows are ordered starting from the smallest N_DESCARGA value to the largest.

In [14]:
import pandas as pd

# Load the dataset
input_file_path = "../data/PARAMETROS_TJ2.csv"  # Replace with the correct path
output_file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"

# Read the data
data = pd.read_csv(input_file_path, delimiter=";", encoding="latin1")

# Convert N_DESCARGA to numeric, coercing errors to NaN
data['N_DESCARGA'] = pd.to_numeric(data['N_DESCARGA'], errors='coerce')

# Drop rows where N_DESCARGA is NaN (invalid values)
data = data.dropna(subset=['N_DESCARGA'])

# Ensure N_DESCARGA is an integer
data['N_DESCARGA'] = data['N_DESCARGA'].astype(int)

# Sort by N_DESCARGA in ascending order
data_sorted = data.sort_values(by='N_DESCARGA')

# Save the sorted dataset to a new CSV file
data_sorted.to_csv(output_file_path, index=False, sep=";", encoding="latin1")

print(f"Sorted data saved to {output_file_path}")

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/1307042932.py:8: DtypeWarning: Columns (52,55,96,132,133,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(input_file_path, delimiter=";", encoding="latin1")


Sorted data saved to ../data/PARAMETROS_TJ2_ORDENADOS.csv


Next, we will convert this .csv file to a .txt to help with training with a RAG system

MODIFY THE TEXT BELOW TO INCLUDE ONLY THE IMPORTANT FIELDS.

In [24]:
import os
import pandas as pd
import re

def process_csv_to_txt(csv_path, txt_path):
    """
    Processes a CSV file with semicolon delimiters, extracts and cleans relevant data,
    and saves it as a .txt file.

    Args:
        csv_path (str): Path to the input CSV file.
        txt_path (str): Path to save the output .txt file.
    """
    try:
        # Read the CSV file with semicolon as the delimiter
        data = pd.read_csv(csv_path, delimiter=';', encoding='latin1', on_bad_lines='skip')
    except Exception as e:
        print(f"Error reading {csv_path}: {e}")
        return

    # Initialize the text content
    text = ''

    # Iterate over the rows of the CSV
    for index, row in data.iterrows():
        # Extract the necessary fields (customize this based on your analysis needs)
        descarga = row.get('N_DESCARGA', 'N/A')
        fecha = row.get('fecha', 'N/A')
        comentario = row.get('comentarioDesc', 'N/A')
        configuracion = row.get('configuracion', 'N/A')
        energia_diamagnetica = row.get('energia_diamagnetica', 'N/A')

        # Format the extracted data into a readable string
        text += (
            f"Descarga: {descarga}\n"
            f"Fecha: {fecha}\n"
            f"Comentario: {comentario}\n"
            f"Configuración: {configuracion}\n"
            f"Energía Diamagnética: {energia_diamagnetica}\n"
            f"{'-'*40}\n"  # Separator for readability
        )

    # Save the processed text to the output file
    try:
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)
        print(f"Processed data saved to {txt_path}")
    except Exception as e:
        print(f"Error saving to {txt_path}: {e}")


# Input and output paths
csv_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"
txt_folder = "../data/txt_output/"

# Ensure the output folder exists
os.makedirs(txt_folder, exist_ok=True)

# Generate the output .txt file name
txt_path = os.path.join(txt_folder, "PARAMETROS_TJ2_ORDENADOS.txt")

# Process the CSV and save it as a .txt file
process_csv_to_txt(csv_path, txt_path)

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/1245340582.py:16: DtypeWarning: Columns (52,54,55,96,105,108,110,122,125,129,132,133,137,138,143,144,145,150,151,154) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_path, delimiter=';', encoding='latin1', on_bad_lines='skip')


Processed data saved to ../data/txt_output/PARAMETROS_TJ2_ORDENADOS.txt


Hay que borrar los datos repetidos, hay muchos donde la hora y la fecha son iguales, aunque el numero de descarga sea diferente, así que habría que dejar solo uno. 

Ahora hay que crear una base de datos vectorial a partir de los archivos .txt (database_creator)

the code is using a GPU if one is available, and it’s enabled by the SentenceTransformer library

TRY THIS IN GOOGLE COLAB TO CREATE THE VECTOR_DATABASE

# Step 2: Import Libraries
import json
import os
import faiss
from sentence_transformers import SentenceTransformer

# Ensure CUDA configuration for PyTorch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Step 3: Define Helper Functions
# Load and split the text file into paragraphs
def load_and_split_text(file_path, context_size=0):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Split the text into paragraphs based on double line breaks
    split_paragraphs = [para.strip() for para in text.split('\n\n') if para.strip()]

    paragraphs = []
    for para in split_paragraphs:
        # Handle missing or invalid values
        para = para.replace("nan", "N/A")  # Replace 'nan' with a placeholder
        paragraphs.append(para)

    # Add context if needed
    if context_size > 0:
        paragraphs_with_context = []
        for i, para in enumerate(paragraphs):
            previous_context = paragraphs[i - 1][-context_size:] if i > 0 else ""
            next_context = paragraphs[i + 1][:context_size] if i < len(paragraphs) - 1 else ""
            combined_para = f"{previous_context} {para} {next_context}"
            paragraphs_with_context.append(combined_para)
        return paragraphs_with_context

    return paragraphs


# Generate embeddings for the paragraphs
def generate_embeddings(paragraphs, model_name):
    model = SentenceTransformer(model_name, device='cuda')
    metadata = []

    for i, para in enumerate(paragraphs):
        metadata.append({
            "id": i,
            "paragraph": para,
        })
    embeddings = model.encode(paragraphs, convert_to_numpy=True)
    return embeddings, metadata


# Build a FAISS index
def build_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index


# Main processing for the nuclear fusion project
def process_nuclear_fusion_data(txt_path, model_name, context_sizes, database_base_path):
    for context_size in context_sizes:
        database_path = os.path.join(database_base_path, f"{model_name}_{context_size}")

        index_path = os.path.join(database_path, "db.index")
        metadata_path = os.path.join(database_path, "metadata.json")

        print(f"Processing with context size {context_size}...")

        # Load and split the text
        print("Loading and splitting text...")
        paragraphs = load_and_split_text(txt_path, context_size=context_size)

        # Generate embeddings
        print("Generating embeddings...")
        embeddings, metadata = generate_embeddings(paragraphs, model_name=model_name)

        # Build FAISS index
        print("Building FAISS index...")
        index = build_faiss_index(embeddings)

        # Save the database
        print(f"Saving the database to {database_path}...")
        if not os.path.exists(database_path):
            os.makedirs(database_path, exist_ok=True)
        faiss.write_index(index, index_path)

        # Save the metadata as a JSON file
        with open(metadata_path, "w", encoding="utf-8") as f:
            json.dump(metadata, f, ensure_ascii=False, indent=4)

    print("Database creation complete.")

# Step 4: Paths and Configurations
TXT_FILE_PATH = "/content/PARAMETROS_TJ2_ORDENADOS.txt"  # Absolute path to your file
MODEL_NAME = "allenai/scibert_scivocab_uncased"  # Use a suitable pre-trained model
CONTEXT_SIZES = [0, 25, 50, 75, 100]  # Adjust context sizes as needed
DATABASE_BASE_PATH = "./vector_database"  # Local path in Colab environment

# Step 5: Process the Text File and Create the Vectorial Database
process_nuclear_fusion_data(TXT_FILE_PATH, MODEL_NAME, CONTEXT_SIZES, DATABASE_BASE_PATH)

# Step 6: Download Output Files
# Zip the output folder for download
!zip -r vector_database.zip ./vector_database

# Provide a link to download
from google.colab import files
files.download('vector_database.zip')

In [2]:
import csv
import json
from collections import OrderedDict

# Path to the input CSV file
csv_file_path = "../data/PARAMETROS_TJ2_model.csv"
# Path to the output JSON file
json_file_path = "../data/PARAMETROS_TJ2_model.json"

# Open and process the CSV file
with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=';')  # Adjust delimiter based on your file
    data = []

    # Read each row and convert to OrderedDict to maintain order
    for row in csv_reader:
        ordered_row = OrderedDict(row)
        data.append(ordered_row)

# Write the data to a JSON file
with open(json_file_path, mode='w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print(f"CSV has been successfully converted to JSON and saved to {json_file_path}")

CSV has been successfully converted to JSON and saved to ../data/PARAMETROS_TJ2_model.json


write a script so that it goes through the json file, and so that if the value is empty, it deletes it. e.g., here it should detele this: 

In [3]:
import json

# Path to the input JSON file
json_file_path = "../data/PARAMETROS_TJ2_model.json"
# Path to the output cleaned JSON file
cleaned_json_file_path = "../data/PARAMETROS_TJ2_model_cleaned.json"

# Open and load the JSON file
with open(json_file_path, mode='r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Function to clean empty values from a dictionary
def clean_empty_values(item):
    if isinstance(item, dict):
        return {k: clean_empty_values(v) for k, v in item.items() if v not in [None, "", []]}
    elif isinstance(item, list):
        return [clean_empty_values(i) for i in item if i not in [None, "", []]]
    else:
        return item

# Clean the data
cleaned_data = clean_empty_values(data)

# Save the cleaned data back to a JSON file
with open(cleaned_json_file_path, mode='w', encoding='utf-8') as cleaned_json_file:
    json.dump(cleaned_data, cleaned_json_file, ensure_ascii=False, indent=4)

print(f"Cleaned JSON has been saved to {cleaned_json_file_path}")

Cleaned JSON has been saved to ../data/PARAMETROS_TJ2_model_cleaned.json


In [ ]:
import json

# Path to the input JSON file
json_file_path = "../data/PARAMETROS_TJ2_model_cleaned.json"

# List of keys to check
keys_to_check = [
    "valvula_li1", "valvula_li2", "valvula_he", "valvula_cx1", "valvula_cx2", 
    "valvula_nb", "valvula_cnb1", "valvula_cnb2", 
    "polaridad_limitador_a3bot", "polaridad_limitador_c3bot", "polaridad_electrodo_a7top"
]

# Open and load the JSON file
with open(json_file_path, mode='r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Find N_DESCARGA values where any of the keys have a value not equal to 0
n_descarga_values = [
    item["N_DESCARGA"] for item in data
    if any(key in item and item[key] not in ["0", 0, None] for key in keys_to_check)
]

# Output the results
print("N_DESCARGA values where any specified key is not 0:")
print(n_descarga_values)

N_DESCARGA values where any specified key is not 0:
['8520', '8521', '8523', '8524', '8525', '8526', '8527', '8528', '8529', '8530', '8531', '8532', '8533', '8534', '8536', '8537', '8538', '8539', '8540', '8541', '8542', '8543', '8544', '8545', '8546', '8547', '8548', '8549', '8550', '8551', '8552', '8553', '8554', '8555', '8556', '8557', '8558', '8559', '8560', '8561', '8562', '8672', '8673', '8674', '8675', '8676', '8677', '8678', '8679', '8688', '8689', '8717', '8718', '8719', '8720', '8721', '9414', '9422', '9423', '9424', '9426', '9427', '9428', '9429', '9430', '9431', '9432', '9433', '9434', '9435', '9436', '9437', '9438', '9439', '9440', '9441', '9442', '9443', '9444', '9445', '9446', '9447', '9448', '9620', '9621', '9622', '9623', '9624', '9833', '9834', '9835', '9836', '9837', '9888', '9889', '9891', '9892', '9893', '9894', '9895', '9898', '9899', '9900', '9901', '9902', '9903', '9904', '9905', '9906', '9907', '9908', '9909', '11669', '11670', '11671', '11672', '14000', '14001

Replace all valvula_* keys with a single valvula = 0 and all polaridad_* keys with polaridad = 0 in the JSON file. This simplifies the structure and reduces redundancy.

In [9]:
import json

# Path to the input JSON file
json_file_path = "../data/PARAMETROS_TJ2_model_cleaned.json"
# Path to the output JSON file
output_json_file_path = "../data/PARAMETROS_TJ2_model_reduced.json"

# Lists of keys to consolidate
valvula_keys = [
    "valvula_li1", "valvula_li2", "valvula_he", "valvula_cx1", "valvula_cx2",
    "valvula_nb", "valvula_cnb1", "valvula_cnb2"
]
polaridad_keys = [
    "polaridad_limitador_a3bot", "polaridad_limitador_c3bot", "polaridad_electrodo_a7top"
]

# Open and load the JSON file
with open(json_file_path, mode='r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Update each item in the JSON data
for item in data:
    # Replace all valvula_* keys with valvula = 0
    if all(item.get(key) in ["0", 0, None] for key in valvula_keys):
        # Remove individual valvula_* keys
        for key in valvula_keys:
            item.pop(key, None)
        # Add consolidated valvula = 0
        item["valvula"] = "0"

    # Replace all polaridad_* keys with polaridad = 0
    if all(item.get(key) in ["0", 0, None] for key in polaridad_keys):
        # Remove individual polaridad_* keys
        for key in polaridad_keys:
            item.pop(key, None)
        # Add consolidated polaridad = 0
        item["polaridad"] = "0"

# Save the updated JSON data to a new file
with open(output_json_file_path, mode='w', encoding='utf-8') as output_json_file:
    json.dump(data, output_json_file, ensure_ascii=False, indent=4)

print(f"Reduced JSON has been saved to {output_json_file_path}")

Reduced JSON has been saved to ../data/PARAMETROS_TJ2_model_reduced.json
